## What are autoencoders?

- In classical machine learning, we introduced PCA as a dimensionality reduction algorithm

- In deep learning auto-encoder has the same purpose as PCA does for tabular data

- "Autoencoding" is a data compression and decompression algorithm where the compression and decompression functions are implemented with neural networks

- Two interesting practical applications of autoencoders are data denoising (which we feature later in this post), and dimensionality reduction

<img src="auto_encoder.png" width="500" height="500">


### Build a MLP based auto-encoder for MNIST

- Write a single fully-connected neural layer as encoder and as decoder with Keras

In [1]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# configure our model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer:

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# prepare our input data. We're using MNIST digits, and we're discarding the labels 
# since we're only interested in encoding/decoding the input images


(x_train, _), (x_test, _) = mnist.load_data()

# normalize all values between 0 and 1 and we will flatten the 28x28 images into vectors of size 784

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Reshape x_train and x_test
x_train = np.reshape(x_train,[-1, 28*28])
x_test = np.reshape(x_test,[-1, 28*28])

# train our autoencoder for 50 epochs

autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))


/Users/miladtoutounchian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 1s 25us/step - loss: 0.3538 - val_loss: 0.2715
Epoch 2/50
60000/60000 [==============================] - 1s 21us/step - loss: 0.2652 - val_loss: 0.2554
Epoch 3/50
60000/60000 [==============================] - 1s 21us/step - loss: 0.2450 - val_loss: 0.2323
Epoch 4/50
60000/60000 [==============================] - 1s 21us/step - loss: 0.2232 - val_loss: 0.2122
Epoch 5/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.2063 - val_loss: 0.1981
Epoch 6/50
60000/60000 [==============================] - 1s 21us/step - loss: 0.1943 - val_loss: 0.1881
Epoch 7/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.1855 - val_loss: 0.1805
Epoch 8/50
60000/60000 [==============================] - 1s 21us/step - loss: 0.1785 - val_loss: 0.1740
Epoch 9/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.1727 - val_loss: 0.1687
Epoch

## If we want to visualize the reconstructed inputs and the encoded representations, what should we do?

In [ ]:
# Solution:

# We should create a separate encoder model

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# As well as the decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

## We can have deep auto-encoder and convolutional auto-encoder

https://blog.keras.io/building-autoencoders-in-keras.html

# Change the cost function to MSE or Keep the binary Cross Entropy 

- For AutoEncoder, we can not just simply do `model.evaluate` and check the accuracy. Maybe the `loss = mse` would be good. We can verify the model performance by human interaction (visualy see the output of auto-encoder images) or feeding the output of AutoEncoder to a trained MLP or trained CNN to see how it works


In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np
from keras import backend as K
from sklearn.metrics import log_loss
import tensorflow as tf

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784, ))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
auto_encoder = Model(input_img, decoded)

(x_train, _), (x_test, _) = mnist.load_data()

# normalize all values between 0 and 1 and we will flatten the 28x28 images into vectors of size 784


x_train = x_train.reshape(x_train.shape[0], 28*28)
x_train = x_train/255



auto_encoder.compile(optimizer='adadelta', loss='binary_crossentropy')
auto_encoder.fit(np.array([x_train[0]]), np.array([x_train[0]]), epochs=100, batch_size=1)

y_p = auto_encoder.predict(np.array([x_train[0]]))[0]
y_t = np.array(x_train[0])

# check the 18th row from bottom for both y_p and y_t
# also change loss from binary_crossentropy to mse (mse has good result too)
print(y_p)
print(y_t)


# from keras
def f_k_binary_cross_entropy(y_tr, y_pr):
    return K.mean(K.binary_crossentropy(y_tr, y_pr), axis=-1)


print(K.eval(f_k_binary_cross_entropy(K.constant(y_t), K.constant(y_p))))


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


# def cross_entropy(predictions, targets):
#     N = predictions.shape[0]
#     ce = -np.sum(targets*np.log(predictions))/N
#     return ce


def f_binary_cross(targets, predictions):
    N = predictions.shape[0]
    return -np.sum(targets * np.log(predictions) + (1 - targets) * np.log(1 - predictions))/N


# print(cross_entropy(y_p, y_t))
print(f_binary_cross(y_t, y_p))
# print(log_loss(y_t, y_p))


# cost = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.constant(tf.cast(y_t, tf.float64),
#                                                logits=tf.constant(tf.cast(y_p, tf.float64))))

cost = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.convert_to_tensor(y_t, dtype=tf.float32),

                                               logits=tf.convert_to_tensor(np.log(y_p/(1-y_p)), dtype=tf.float32))
binary_cross_entropy = tf.reduce_mean(cost)
sess = tf.Session()

print(sess.run(binary_cross_entropy))


def f_k_l_divergence(y_tr, y_pr):
    y_true = K.clip(y_tr, K.epsilon(), 1)
    y_pred = K.clip(y_pr, K.epsilon(), 1)
    return K.sum(y_true * K.log(y_true / y_pred), axis=-1)


def KL(P,Q):
    epsilon = 0.0000000001
    # You may want to instead make copies to avoid changing the np arrays.
    P = P+epsilon
    Q = Q+epsilon
    # divergence = np.sum(P*np.log(P/Q))
    divergence = np.sum(P * np.log(P) - P * np.log(Q))
    return divergence


print(K.eval(f_k_l_divergence(K.constant(y_t), K.constant(y_p))))
print(KL(y_t, y_p))
